# Packages

In [1]:
import numpy as np
import pandas as pd
import os

from lightgbm import LGBMRegressor
from dddex.levelSetKDE import LevelSetKDE

In [2]:
path = ('/home/kagu/yaz_trainingData/dataYaz.csv')
data = pd.read_csv(path)

In [3]:
dataTrain = data[data['label'] == 'train']
dataTest = data[data['label'] == 'test']

XTrain = dataTrain.drop(['dayIndex', 'label', 'id', 'demand'], axis = 1)
XTest = dataTest.drop(['dayIndex', 'label', 'id', 'demand'], axis = 1)

YTrain = dataTrain['demand']
YTest = dataTest['demand']

In [13]:
LGBM = LGBMRegressor()
LGBM.fit(X = XTrain.iloc[0:100, :], y = YTrain.iloc[0:100])

LGBMRegressor()

In [14]:
LevelSetKDE(estimator = LGBM, binSize = 10)

LevelSetKDE(estimator = LGBMRegressor(), binSize = 10)

In [15]:
LevelSetKDE.fit(X = XTrain, y = YTrain)

TypeError: fit() missing 1 required positional argument: 'self'

In [17]:
LevelSetKDE.fit??

Signature: LevelSetKDE.fit(self: 'LevelSetKDE', X: 'np.ndarray', y: 'np.ndarray')
Docstring: Fit weights-based predictor on training samples
Source:   
@patch
def fit(self: LevelSetKDE, 
        X: np.ndarray, # Feature matrix used by 'estimator' to predict 'y'.
        y: np.ndarray, # 1-dimensional target variable corresponding to the features 'X'.
        ):

    if self.binSize > y.shape[0]:
        raise ValueError("'binSize' mustn't be bigger than the size of 'y'!")

    yPredTrain = self.estimator.predict(X)

    binPerTrainPred, indicesPerBin = generateBins(binSize = self.binSize,
                                                  yPredTrain = yPredTrain)

    #---

    nn = NearestNeighbors(algorithm = 'kd_tree')
    yPredTrain_reshaped = np.reshape(yPredTrain, newshape = (len(yPredTrain), 1))

    nn.fit(X = yPredTrain_reshaped)

    #---

    self.y = y
    self.yPredTrain = yPredTrain
    self.binPerTrainPred = binPerTrainPred
    self.indicesPerBin = indicesPerBin
    self.